**Experiment for obtaining 24 Hr prediction from Persistence Model in rainymotion library**

Author: Divya S. Vidyadharan

File use: For predicting 24 Hr precipitation images. 

Date Created: 19-03-21

Last Updated: 20-03-21

Python version: 3.8.2

In [9]:
import h5py
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.misc

from rainymotion.models import Persistence
from rainymotion.metrics import *
import os
import cv2
import pandas as pd
import wradlib.ipol as ipol # for interpolation
from rainymotion import metrics
from rainymotion import utils
from scipy.ndimage import map_coordinates
import timeit
#from tvl1sindysupport import tvl1utilities -in future our own library

In [10]:
 times=['0000','0010', '0020', '0030', '0040', '0050',
          '0100', '0110', '0120', '0130', '0140', '0150',
          '0200', '0210', '0220', '0230', '0240', '0250',
          '0300', '0310', '0320', '0330', '0340', '0350',
          '0400', '0410', '0420', '0430', '0440' ,'0450',
          '0500', '0510', '0520', '0530', '0540', '0550',
          
          '0600', '0610', '0620', '0630', '0640', '0650',
          '0700', '0710', '0720', '0730', '0740', '0750',
          '0800', '0810', '0820', '0830', '0840', '0850',
          '0900', '0910', '0920', '0930', '0940', '0950',
          '1000', '1010', '1020', '1030', '1040', '1050',
          '1100', '1110', '1120', '1130', '1140', '1150',
          '1200', '1210', '1220', '1230', '1240', '1250',
          
          
          '1300', '1310', '1320', '1330', '1340', '1350',
          '1400', '1410', '1420', '1430', '1440', '1450',
          '1500', '1510', '1520', '1530', '1540', '1550',
          '1600', '1610', '1620', '1630', '1640', '1650',
          '1700', '1710', '1720', '1730', '1740', '1750',
          '1800', '1810', '1820', '1830', '1840', '1850',
          
          '1900', '1910', '1920', '1930', '1940', '1950',
          '2000', '2010', '2020', '2030', '2040', '2050',
          '2100', '2110', '2120', '2130', '2140', '2150',
          '2200', '2210', '2220', '2230', '2240', '2250',
          '2300', '2310', '2320', '2330', '2340', '2350']


In [35]:
def discrete_cmap(N, base_cmap=None):
    """Create an N-bin discrete colormap from the specified input map"""

    # Note that if base_cmap is a string or None, you can simply do
    #    return plt.cm.get_cmap(base_cmap, N)
    # The following works for string, None, or a colormap instance:

    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)

In [36]:
#For plotting map - currently using function as in source code Need to change to Cartopy
def plotMap(title,img, lat1, lat2, long1, long2, outputPath,last=0):
   
    (height, width) = img.shape

#     print(img.min(), img.max())
    intensity = np.asarray(img, dtype=np.float32)
#     print(intensity.min(), intensity.max())

    #intensity_level = convert_rep_to_level(intensity).reshape(height, width)
#     print(intensity.min(), intensity.max())
    intensity_level = np.flipud(intensity)

    dLon = (long2 - long1) / width
    dLat = (lat2 - lat1) / height

    lon = np.arange(long1, long2, dLon)
    lat = np.arange(lat1, lat2, dLat)
    lons, lats = np.meshgrid(lon, lat)
#     print(lons.shape, lats.shape)

    fig = plt.figure(figsize=(12, 8))

    # Set up Basemap instance
    m = Basemap(projection="cyl",
                llcrnrlon=long1, urcrnrlon=long2,
                llcrnrlat=lat1, urcrnrlat=lat2,
                resolution='h')

    # Add geographic outlines
    m.drawcoastlines(color='black')
    m.drawstates()
    m.drawcountries()
    m.drawmeridians(np.arange(long1, long2, 1), labels=[True, False, False, True])
    m.drawparallels(np.arange(lat1, lat2, 1), labels=[True, False, True, False])
    #m.drawmeridians(np.arange(new_lon_min, new_lon_max, 1), labels=[False, False, False, False])
    #m.drawparallels(np.arange(new_lat_min, new_lat_max, 1), labels=[False, False, False, False])


    # Plot Data
    #cs = m.contourf(lons, lats, intensity_level, shading='flat', levels=list(range(1, 65)), cmap=get_cmap("jet"))
    
    #cs = m.contourf(lons, lats, intensity_level,shading='flat', levels=list(range(1,65)), cmap=get_cmap("gist_earth"))
    cs = m.contourf(lons, lats, intensity_level,shading='flat', levels=list(range(1,65)), cmap=discrete_cmap(8,"jet"))
    # Add Colorbar
    if last==1:
        cb = plt.colorbar(cs ,shrink=1.0) #, extend='both')

    # Add Title
    plt.title(title)
    plt.savefig(outputPath, bbox_inches='tight', pad_inches=0.0)
    plt.close()



In [11]:
# Common Initialization
eventName = "TyphoonFaxai"
eventDate ="20190908"

#Latitude and Longitude of Typhoon Faxai

lat1 = 32.5
lat2 = 39
long1 = 136
long2 = 143


pred_date = 20190908 #YYYYMMDD
[height, width] = [781,561]
eventNameDate = eventName + "_" + eventDate

startHr = 2
startMin= 40

predStartHr = 300
step = 5 #for rainymotion models


# For radar images
inputFolder = "./ForExperiments/Exp1/RadarImages/HeavyRainfall/For300/"
# outputFolder= "./ForExperiments/Exp1/Results/"
# print(inputFolder)

fileType='.bin'
timeStep = 10 # for Japan Radar Data

modelName = "Persistence"

stepRainyMotion = 5 # 5 minutes



##recentFramePath##
recentFrameFolder = str(pred_date)+"_set_24Hr_bin" #20190908_set_24Hr_bin
recentFramePath = "/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/%s"%recentFrameFolder
print ("\n Recent frame path ",recentFramePath)
inputFolder = recentFramePath
print("\n Input folder is ",inputFolder)
##Output path where predicted images for visual comparison are saved.##
outputimgpath = "/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/24hroutputs/%i/%s/%s"%(pred_date,modelName,"pred_images")
os.makedirs(outputimgpath, exist_ok=True)
print ("\n Output image path is ",outputimgpath)

##Output path where evaluation results are saved as csv files.##
outputevalpath = "/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/24hroutputs/%i/%s/%s"%(pred_date,modelName,"eval_results")
os.makedirs(outputevalpath, exist_ok=True)
print ("\n Output eval results in ",outputevalpath)

savepath = outputimgpath#"Outputs/%i/%s"%(pred_date,pred_times[0])

noOfImages = 2#  Model needs 24 frames

step = 5
outputFilePath = outputimgpath+'/'
outputFilePath = outputFilePath + eventNameDate

print(outputFilePath)

hrlimit = len(times)
leadsteps = 6
totinputframes = 2



 Recent frame path  /home/divya/divya/OneFullDayData_7TestCases_WNIMar5/20190908_set_24Hr_bin

 Input folder is  /home/divya/divya/OneFullDayData_7TestCases_WNIMar5/20190908_set_24Hr_bin

 Output image path is  /home/divya/divya/OneFullDayData_7TestCases_WNIMar5/24hroutputs/20190908/Persistence/pred_images

 Output eval results in  /home/divya/divya/OneFullDayData_7TestCases_WNIMar5/24hroutputs/20190908/Persistence/eval_results
/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/24hroutputs/20190908/Persistence/pred_images/TyphoonFaxai_20190908


In [12]:
def gettimes24hr(pred_time):
    
   
   # times=np.array(times)
    inptimes = []
    pred_times = []
    index = times.index(pred_time)
    indexlimit = len(times)
    print("Leadsteps are ", leadsteps)
    if (index+leadsteps) < indexlimit:
        pred_times = times[index:index+leadsteps]
    if (index-totinputframes)>=0:
        inptimes = times[index-totinputframes:index]
              
    print("PredTimes:",pred_times)
    print("InpTimes:",inptimes)
    print("Get Time Success..")
    
    return inptimes, pred_times

In [13]:
def readRadarImages(pred_time,inputpath,height,width, noOfImages,fileType):
    files = (os.listdir(recentFramePath))
    files.sort()
    inputRadarImages = []
    
    i = 0
    index = times.index(pred_time)
#     print(index)
    inputframes = times[index-noOfImages:index]
#     print(len(inputframes))
    while (i<noOfImages):
        
        inputframetime = "_"+inputframes[i]
        i = i +1
        for fileName in files:

            if inputframetime in fileName:
                print("The input image at %s is available",inputframetime)
                print(fileName)
                if fileName.endswith(fileType):

                    
                    inputFileName =recentFramePath+'/'+fileName
                    fd = open(inputFileName,'rb')
                    #print(inputFileName)

                    # straight to numpy data (no buffering) 
                    inputFrame = np.fromfile(fd, dtype = np.dtype('float32'), count = 2*height*width)
                    inputFrame = np.reshape(inputFrame,(height,width))
                    inputFrame = inputFrame.astype('float16') 

                    #print(recentFrame.shape)
                    inputRadarImages.append(inputFrame)
                    #else:
                    #    print("Sorry, unable to find file.")
            
    inputRadarImages = np.stack(inputRadarImages, axis=0)
    print(inputRadarImages.shape)
    return inputRadarImages


**1.2 Persistence**

In [30]:
def doPersistenceNowcasting(startpredtime, saveimages):

    
    model = Persistence()
    model.input_data = readRadarImages(startpredtime,inputFolder,height,width, noOfImages,fileType)
    start = timeit.timeit()
    nowcastPersistence = model.run()
    end = timeit.timeit()
    sparseTime = end - start
    print("Persistence took ",end - start)
    nowcastPersistence.shape
    print("Saving the nowcast images. Please wait...")
    for i in range(leadsteps):
        outFrameName = outputFilePath + '_'+str(predStartHr+(i*5))+'.png'
        #matplotlib.image.imsave(outFrameName, nowcastPersistence[i])
        if i == leadsteps-1:
            last = 1
        else:
            last = 0
        if (saveimages):
            plotMap(modelName+' '+str(predStartHr+(i*5)), nowcastPersistence[i], lat1, lat2, long1, long2, outFrameName,last)
    print("Finished persistence model nowcasting!")
    return nowcastPersistence

**2. Performance Evaluation**

In [15]:

def getGroundTruthImages(pred_times,leadsteps,recentFramePath,height,width,fileType):
    files = (os.listdir(recentFramePath))
    files.sort()
    groundTruthImages = []
    
    i = 0
    while (i<leadsteps):
        
        groundtruthtime = "_"+pred_times[i]
        i = i +1
        for fileName in files:

            if groundtruthtime in fileName:
                print("The ground truth at %s is available",groundtruthtime)
                print(fileName)
                if fileName.endswith(fileType):

                    
                    inputFileName =recentFramePath+'/'+fileName
                    fd = open(inputFileName,'rb')
                    #print(inputFileName)

                    # straight to numpy data (no buffering) 
                    recentFrame = np.fromfile(fd, dtype = np.dtype('float32'), count = 2*height*width)
                    recentFrame = np.reshape(recentFrame,(height,width))
                    recentFrame = recentFrame.astype('float16') 

                    #print(recentFrame.shape)
                    groundTruthImages.append(recentFrame)
                    #else:
                    #    print("Sorry, unable to find file.")
            
    groundTruthImages = np.moveaxis(np.dstack(groundTruthImages), -1, 0)
    #print(groundTruthImages.shape)
    return groundTruthImages

In [16]:
def evaluate(nowcasts):
    fileType = '.bin'
    leadsteps = 6 # 6 for 1 hr prediction, 18 for 3hr prediction
    groundTruthPath = recentFramePath
    print(pred_times)
    groundTruthImgs = getGroundTruthImages(pred_times,leadsteps,groundTruthPath,height,width,fileType)
  
    maelist = []
    farlist = []
    podlist= []
    csilist= []
    thres =1.0
    
    noOfPrecipitationImages = 6
    j = 0  # using another index to skip 5min interval data from rainymotion
    for i in range(noOfPrecipitationImages):
        mae = MAE(groundTruthImgs[i],nowcasts[j])
        far = FAR(groundTruthImgs[i],nowcasts[j], threshold=0.1)
        pod = POD(groundTruthImgs[i],nowcasts[j], threshold=0.1)
        csi = CSI(groundTruthImgs[i],nowcasts[j],thres)
        maelist.append(mae)
        farlist.append(far)
        podlist.append(pod)
        csilist.append(csi)
        j = j + 2
    return csilist,maelist,farlist,podlist


    
    

**2. 24 Hr Prediction**

In [38]:
startpredtime = '0100' #'1100'

index = times.index(startpredtime)
indexlimit = times.index('2250') # Since we have only 6 more ground truths available from this time
print(index)
print("Last prediction is at index ", indexlimit)
csilist = []
maelist = []
podlist = []
farlist = []
pred_time = startpredtime

while index<indexlimit:#len(times):
    print(times[index])
    saveimages = 0
#     if (index==66):
#         saveimages=1
    intimes, pred_times = gettimes24hr(pred_time)
    nowcasts = doPersistenceNowcasting(pred_time,saveimages)
    
    csi,mae,far,pod = evaluate(nowcasts)
    
    csilist.append(csi)
    maelist.append(mae)
    podlist.append(pod)
    farlist.append(far)
    
    index = index+1
    pred_time = times[index] 
print("Successfully completed persistence nowcasting!")    

6
Last prediction is at index  137
0100
Leadsteps are  6
PredTimes: ['0100', '0110', '0120', '0130', '0140', '0150']
InpTimes: ['0040', '0050']
Get Time Success..
The input image at %s is available _0040
20190908_004000.000.bin
The input image at %s is available _0050
20190908_005000.000.bin
(2, 781, 561)
Persistence took  -0.004249449004419148
Saving the nowcast images. Please wait...
Finished persistence model nowcasting!
['0100', '0110', '0120', '0130', '0140', '0150']
The ground truth at %s is available _0100
20190908_010000.000.bin
The ground truth at %s is available _0110
20190908_011000.000.bin
The ground truth at %s is available _0120
20190908_012000.000.bin
The ground truth at %s is available _0130
20190908_013000.000.bin
The ground truth at %s is available _0140
20190908_014000.000.bin
The ground truth at %s is available _0150
20190908_015000.000.bin
0110
Leadsteps are  6
PredTimes: ['0110', '0120', '0130', '0140', '0150', '0200']
InpTimes: ['0050', '0100']
Get Time Success..

0240
Leadsteps are  6
PredTimes: ['0240', '0250', '0300', '0310', '0320', '0330']
InpTimes: ['0220', '0230']
Get Time Success..
The input image at %s is available _0220
20190908_022000.000.bin
The input image at %s is available _0230
20190908_023000.000.bin
(2, 781, 561)
Persistence took  -0.0005435920029412955
Saving the nowcast images. Please wait...
Finished persistence model nowcasting!
['0240', '0250', '0300', '0310', '0320', '0330']
The ground truth at %s is available _0240
20190908_024000.000.bin
The ground truth at %s is available _0250
20190908_025000.000.bin
The ground truth at %s is available _0300
20190908_030000.000.bin
The ground truth at %s is available _0310
20190908_031000.000.bin
The ground truth at %s is available _0320
20190908_032000.000.bin
The ground truth at %s is available _0330
20190908_033000.000.bin
0250
Leadsteps are  6
PredTimes: ['0250', '0300', '0310', '0320', '0330', '0340']
InpTimes: ['0230', '0240']
Get Time Success..
The input image at %s is availabl

0420
Leadsteps are  6
PredTimes: ['0420', '0430', '0440', '0450', '0500', '0510']
InpTimes: ['0400', '0410']
Get Time Success..
The input image at %s is available _0400
20190908_040000.000.bin
The input image at %s is available _0410
20190908_041000.000.bin
(2, 781, 561)
Persistence took  -0.00032163099967874587
Saving the nowcast images. Please wait...
Finished persistence model nowcasting!
['0420', '0430', '0440', '0450', '0500', '0510']
The ground truth at %s is available _0420
20190908_042000.000.bin
The ground truth at %s is available _0430
20190908_043000.000.bin
The ground truth at %s is available _0440
20190908_044000.000.bin
The ground truth at %s is available _0450
20190908_045000.000.bin
The ground truth at %s is available _0500
20190908_050000.000.bin
The ground truth at %s is available _0510
20190908_051000.000.bin
0430
Leadsteps are  6
PredTimes: ['0430', '0440', '0450', '0500', '0510', '0520']
InpTimes: ['0410', '0420']
Get Time Success..
The input image at %s is availab

0600
Leadsteps are  6
PredTimes: ['0600', '0610', '0620', '0630', '0640', '0650']
InpTimes: ['0540', '0550']
Get Time Success..
The input image at %s is available _0540
20190908_054000.000.bin
The input image at %s is available _0550
20190908_055000.000.bin
(2, 781, 561)
Persistence took  -0.0003659290086943656
Saving the nowcast images. Please wait...
Finished persistence model nowcasting!
['0600', '0610', '0620', '0630', '0640', '0650']
The ground truth at %s is available _0600
20190908_060000.000.bin
The ground truth at %s is available _0610
20190908_061000.000.bin
The ground truth at %s is available _0620
20190908_062000.000.bin
The ground truth at %s is available _0630
20190908_063000.000.bin
The ground truth at %s is available _0640
20190908_064000.000.bin
The ground truth at %s is available _0650
20190908_065000.000.bin
0610
Leadsteps are  6
PredTimes: ['0610', '0620', '0630', '0640', '0650', '0700']
InpTimes: ['0550', '0600']
Get Time Success..
The input image at %s is availabl

0740
Leadsteps are  6
PredTimes: ['0740', '0750', '0800', '0810', '0820', '0830']
InpTimes: ['0720', '0730']
Get Time Success..
The input image at %s is available _0720
20190908_072000.000.bin
The input image at %s is available _0730
20190908_073000.000.bin
(2, 781, 561)
Persistence took  -0.0005823189858347178
Saving the nowcast images. Please wait...
Finished persistence model nowcasting!
['0740', '0750', '0800', '0810', '0820', '0830']
The ground truth at %s is available _0740
20190908_074000.000.bin
The ground truth at %s is available _0750
20190908_075000.000.bin
The ground truth at %s is available _0800
20190908_080000.000.bin
The ground truth at %s is available _0810
20190908_081000.000.bin
The ground truth at %s is available _0820
20190908_082000.000.bin
The ground truth at %s is available _0830
20190908_083000.000.bin
0750
Leadsteps are  6
PredTimes: ['0750', '0800', '0810', '0820', '0830', '0840']
InpTimes: ['0730', '0740']
Get Time Success..
The input image at %s is availabl

...

1600
Leadsteps are  6
PredTimes: ['1600', '1610', '1620', '1630', '1640', '1650']
InpTimes: ['1540', '1550']
Get Time Success..
The input image at %s is available _1540
20190908_154000.000.bin
The input image at %s is available _1550
20190908_155000.000.bin
(2, 781, 561)
Persistence took  -0.0007965630211401731
Saving the nowcast images. Please wait...
Finished persistence model nowcasting!
['1600', '1610', '1620', '1630', '1640', '1650']
The ground truth at %s is available _1600
20190908_160000.000.bin
The ground truth at %s is available _1610
20190908_161000.000.bin
The ground truth at %s is available _1620
20190908_162000.000.bin
The ground truth at %s is available _1630
20190908_163000.000.bin
The ground truth at %s is available _1640
20190908_164000.000.bin
The ground truth at %s is available _1650
20190908_165000.000.bin
1610
Leadsteps are  6
PredTimes: ['1610', '1620', '1630', '1640', '1650', '1700']
InpTimes: ['1550', '1600']
Get Time Success..
The input image at %s is availabl

1740
Leadsteps are  6
PredTimes: ['1740', '1750', '1800', '1810', '1820', '1830']
InpTimes: ['1720', '1730']
Get Time Success..
The input image at %s is available _1720
20190908_172000.000.bin
The input image at %s is available _1730
20190908_173000.000.bin
(2, 781, 561)
Persistence took  -0.0004086370172444731
Saving the nowcast images. Please wait...
Finished persistence model nowcasting!
['1740', '1750', '1800', '1810', '1820', '1830']
The ground truth at %s is available _1740
20190908_174000.000.bin
The ground truth at %s is available _1750
20190908_175000.000.bin
The ground truth at %s is available _1800
20190908_180000.000.bin
The ground truth at %s is available _1810
20190908_181000.000.bin
The ground truth at %s is available _1820
20190908_182000.000.bin
The ground truth at %s is available _1830
20190908_183000.000.bin
1750
Leadsteps are  6
PredTimes: ['1750', '1800', '1810', '1820', '1830', '1840']
InpTimes: ['1730', '1740']
Get Time Success..
The input image at %s is availabl

1920
Leadsteps are  6
PredTimes: ['1920', '1930', '1940', '1950', '2000', '2010']
InpTimes: ['1900', '1910']
Get Time Success..
The input image at %s is available _1900
20190908_190000.000.bin
The input image at %s is available _1910
20190908_191000.000.bin
(2, 781, 561)
Persistence took  -0.0003601810021791607
Saving the nowcast images. Please wait...
Finished persistence model nowcasting!
['1920', '1930', '1940', '1950', '2000', '2010']
The ground truth at %s is available _1920
20190908_192000.000.bin
The ground truth at %s is available _1930
20190908_193000.000.bin
The ground truth at %s is available _1940
20190908_194000.000.bin
The ground truth at %s is available _1950
20190908_195000.000.bin
The ground truth at %s is available _2000
20190908_200000.000.bin
The ground truth at %s is available _2010
20190908_201000.000.bin
1930
Leadsteps are  6
PredTimes: ['1930', '1940', '1950', '2000', '2010', '2020']
InpTimes: ['1910', '1920']
Get Time Success..
The input image at %s is availabl

2100
Leadsteps are  6
PredTimes: ['2100', '2110', '2120', '2130', '2140', '2150']
InpTimes: ['2040', '2050']
Get Time Success..
The input image at %s is available _2040
20190908_204000.000.bin
The input image at %s is available _2050
20190908_205000.000.bin
(2, 781, 561)
Persistence took  -0.0006460269796662033
Saving the nowcast images. Please wait...
Finished persistence model nowcasting!
['2100', '2110', '2120', '2130', '2140', '2150']
The ground truth at %s is available _2100
20190908_210000.000.bin
The ground truth at %s is available _2110
20190908_211000.000.bin
The ground truth at %s is available _2120
20190908_212000.000.bin
The ground truth at %s is available _2130
20190908_213000.000.bin
The ground truth at %s is available _2140
20190908_214000.000.bin
The ground truth at %s is available _2150
20190908_215000.000.bin
2110
Leadsteps are  6
PredTimes: ['2110', '2120', '2130', '2140', '2150', '2200']
InpTimes: ['2050', '2100']
Get Time Success..
The input image at %s is availabl

2240
Leadsteps are  6
PredTimes: ['2240', '2250', '2300', '2310', '2320', '2330']
InpTimes: ['2220', '2230']
Get Time Success..
The input image at %s is available _2220
20190908_222000.000.bin
The input image at %s is available _2230
20190908_223000.000.bin
(2, 781, 561)
Persistence took  -0.00036697901668958366
Saving the nowcast images. Please wait...
Finished persistence model nowcasting!
['2240', '2250', '2300', '2310', '2320', '2330']
The ground truth at %s is available _2240
20190908_224000.000.bin
The ground truth at %s is available _2250
20190908_225000.000.bin
The ground truth at %s is available _2300
20190908_230000.000.bin
The ground truth at %s is available _2310
20190908_231000.000.bin
The ground truth at %s is available _2320
20190908_232000.000.bin
The ground truth at %s is available _2330
20190908_233000.000.bin
Successfully completed persistence nowcasting!


In [39]:
# For debugging
print(len(maelist))
print("\n\n")
print(len(csilist))
print("\n\n")
print(len(podlist))
print("\n\n")
print(len(farlist))


131



131



131



131


**To save results in excel workbook**

In [40]:
import xlwt 
from xlwt import Workbook 
  
# Workbook is created 
wb = Workbook() 


In [41]:
def writeinexcelsheet(sheetname, wb, results):
     
    sheet1 = wb.add_sheet(sheetname) 
    sheet1.write(0, 0, 'Pred.no.') 
    sheet1.write(0, 1, 't (pred start time)') 
    sheet1.write(0, 2, 't + 10') 
    sheet1.write(0, 3, 't + 20') 
    sheet1.write(0, 4, 't + 30') 
    sheet1.write(0, 5, 't + 40') 
    sheet1.write(0, 6, 't + 50') 
    
    
    col = 0

    rows = len(results)
    cols = len(results[0])
    print(cols)
    for rowno in range(rows):
        
        sheet1.write(rowno+1,0,rowno+1)   
        for col in range(cols):
#             print(rowno+1,col+1,results[rowno][col])
            sheet1.write(rowno+1,col+1,results[rowno][col].astype('float64'))
               
#          sheet1.write(row, col, str(data))
#          print(row,col,data)
    
    
    

In [42]:
writeinexcelsheet('CSI',wb,csilist)
writeinexcelsheet('MAE',wb,maelist)
writeinexcelsheet('FAR',wb,farlist)
writeinexcelsheet('POD',wb,podlist)
excelpath = "/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/24hroutputs/20190908/Persistence/eval_results/"
excelpath = excelpath + 'resultsPersistence.xls'
wb.save(excelpath)


6
6
6
6
